# Clustering and Displaying Geospatial data using Uber H3

In [1]:
import numpy as np
import pandas as pd
import folium
import webbrowser
import os
import math

from h3 import h3
from folium import Map

## Helper Functions

The `load_data` function loads two year's worth of UK accidents data into a DataFrame.

In [2]:
def load_data():
    column_types = {'Accident_Index': np.string_, 'LSOA_of_Accident_Location': np.string_}
    uk2015 = pd.read_csv("data/DfTRoadSafety_Accidents_2015.csv", dtype=column_types)
    uk2016 = pd.read_csv("data/dftRoadSafety_Accidents_2016.csv", dtype=column_types)
    return uk2015.append(uk2016)

The `create_map` function creates and populates a folium map from the given `clusters` dictionary. Each value of the dictionary is also a dictionary with two recognizable keys: `geom` and `count`. The value associated with the `geom` key is the list of locations of the hexagon's vertices. The value associated with the `count` key contains the number of accidents counted within this shape.

In [3]:
def create_map(clusters):
    # Create the map object
    map = Map(tiles="cartodbpositron", 
          attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        tooltip = "{0} accidents".format(cluster['count'])
        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#ff0000', 
                                               fill_color='#ff0000', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map)

    # Determine the map bounding box
    max_lat = df.Latitude.max()
    min_lat = df.Latitude.min()
    max_lon = df.Longitude.max()
    min_lon = df.Longitude.min()
    
    # Fit the map to the bounds
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map

The `show_map` function saves the HTML generated by the map into a file and then opens a new browser tab with its contents.

In [4]:
def show_map(map, file_name):
    map.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)

# Data Loading and Cleaning
Start by loading the data files for 2015 and 2016. Please note that more data files are available on the `data` folder.

In [5]:
uk_acc = load_data()

Drop invalid rows

In [6]:
uk_acc.dropna()

uk_acc = uk_acc.loc[uk_acc.Latitude <=  90.0]
uk_acc = uk_acc.loc[uk_acc.Latitude >= -90.0]

uk_acc = uk_acc.loc[uk_acc.Longitude <=  180.0]
uk_acc = uk_acc.loc[uk_acc.Longitude >= -180.0]

Create the radian longitude and latitude columns

In [7]:
uk_acc['rad_lng'] = np.radians(uk_acc['Longitude'].to_numpy())
uk_acc['rad_lat'] = np.radians(uk_acc['Latitude'].to_numpy())

Set the DBSCAN parameters

In [8]:
eps_in_meters = 50.0
num_samples = 10

Cluster the data

In [9]:
from sklearn.cluster import DBSCAN

earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

uk_acc['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine').fit_predict(uk_acc[['rad_lat', 'rad_lng']])

Select the detail level for the H3 tiles. See the official table [here](https://uber.github.io/h3/#/documentation/core-library/resolution-table).

In [10]:
h3_level = 11

The function `lat_lng_to_h3` converts a location's coordinates into an H3 key of the preset level.

In [11]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['Latitude'], row['Longitude'], h3_level)

Create a new DataFrame column with the H3 encoded key for the given latitude and longitude pair.

In [12]:
uk_acc['h3'] = uk_acc.apply(lat_lng_to_h3, axis=1)

Select only the locations that belong to a DBSCAN-generated cluster. Clusters marked with -1 are noise.

In [13]:
df = uk_acc[uk_acc.cluster != -1].copy()

Create the cluster dictionary. Note that we are just collecting the locations that belong in a cluster, _irrespective of the cluster_.

In [14]:
clusters = dict()

for index, row in df.iterrows():
    key = row['h3']
    if key in clusters:
        clusters[key]['count'] += 1
    else:
        clusters[key] = {"count": 1,
                         "geom": h3.h3_to_geo_boundary(h3_address=key)}

In [17]:
uk_acc.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,rad_lng,rad_lat,cluster,h3
0,201501BS70001,525130.0,180050.0,-0.198465,51.505538,1,3,1,1,12/01/2015,...,1,0,0,1,1,E01002825,-0.003464,0.898941,-1,8b194ada4aa4fff
1,201501BS70002,526530.0,178560.0,-0.178838,51.491836,1,3,1,1,12/01/2015,...,1,0,0,1,1,E01002820,-0.003121,0.898702,-1,8b194ada58eafff
2,201501BS70004,524610.0,181080.0,-0.205590,51.514910,1,3,1,1,12/01/2015,...,2,0,0,1,1,E01002833,-0.003588,0.899105,-1,8b194ada4d6efff
3,201501BS70005,524420.0,181080.0,-0.208327,51.514952,1,3,1,1,13/01/2015,...,2,0,0,1,2,E01002874,-0.003636,0.899106,-1,8b194ada4d50fff
4,201501BS70008,524630.0,179040.0,-0.206022,51.496572,1,2,2,1,09/01/2015,...,2,0,0,1,2,E01002814,-0.003596,0.898785,20,8b194ada42d0fff


In [21]:
int("8b194ada42d0fff", 16)

626445296685879295

Create and show the map from the clusters created above.

In [ ]:
map = create_map(clusters)

In [ ]:
show_map(map, "map_{0}.html".format(h3_level))

# Clustering with H3
Clustering with H3 is easy. Just iterate through all points and aggregate the hit counts for each key. Here we don't care about noise, all points are good.

In [ ]:
h3_clusters = dict()

for index, row in uk_acc.iterrows():
    key = row['h3']
    if key in h3_clusters:
        h3_clusters[key]["count"] += 1
    else:
        h3_clusters[key] = {"count": 1,
                            "geom": h3.h3_to_geo_boundary(h3_address=key)}

We will only be displaying clusters with at least five observations (an arbitrary number, of course). Restricting the minimum count helps limit the number of hexagons that are displayed, which might have an impact on the map performance.

In [ ]:
relevant_clusters = { key:value for (key,value) in h3_clusters.items() if value['count'] >= 5}

And that's it! We can now show the map with the H3-generated clusters.

In [ ]:
h3_map = create_map(relevant_clusters)

In [ ]:
show_map(h3_map, "map_h3_{0}.html".format(h3_level))